# Cold-Start Recommender: Colab Workflow

Run the cells in order to clone the PGMS cold-start benchmark, download the requested dataset (MovieLens, Amazon, Goodreads, or Microsoft News), prepare leakage-safe warm/cold splits, and train every baseline plus CMCL (default + ablations). Upload only this notebook to Colab; it will set up everything inside `/content/PGMS_for_Recommender_Systems` and save artifacts under `coldstart/output/notebook_run_*`.

In [ ]:
from pathlib import Path

PROJECT_URL = "https://github.com/mohdfaour03/PGMS_for_Recommender_Systems.git"
PROJECT_ROOT = Path("/content/PGMS_for_Recommender_Systems")

if not PROJECT_ROOT.exists():
    !git clone $PROJECT_URL $PROJECT_ROOT
else:
    print(f"Repository already present at {PROJECT_ROOT}")

%cd /content/PGMS_for_Recommender_Systems

## Configure the run
Choose the source dataset (MovieLens, Amazon, or Goodreads), adjust the interaction cap if you need to conserve RAM, and set the random seed / evaluation cutoffs. The cell below also locates the project root inside Colab and loads the YAML defaults.

> **Heads up (Microsoft News)**: the MIND dataset requires accepting Microsoft's license. If you set `DATA_SOURCE = "msnews"`, download the requested zip (e.g., `MINDsmall_train.zip`) from [msnews.github.io](https://msnews.github.io/) and place it under `coldstart/data/` before running the setup cell. The notebook will reuse the local file and extract it automatically.

In [ ]:
from datetime import datetime
from pathlib import Path
import sys


def locate_project_root() -> Path:
    current = Path.cwd().resolve()
    for candidate in (current, *current.parents):
        if (candidate / "coldstart" / "src").exists():
            return candidate
    raise RuntimeError("Cannot locate project root containing 'coldstart/src'.")


PROJECT_ROOT = locate_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from coldstart.src.notebook_utils import (
    _read_simple_yaml,
    build_amazon_interaction_frame,
    build_goodreads_interaction_frame,
    build_interaction_frame,
    build_msnews_interaction_frame,
    GOODREADS_GENRES,
    MSNEWS_VARIANTS,
)
from coldstart.src import data_io, pipeline

DATA_SOURCE = "movielens"  # options: 'movielens', 'amazon', 'goodreads', 'msnews'
DATASET_VARIANT = "medium"  # MovieLens-only
AMAZON_VARIANT = "beauty"
GOODREADS_VARIANT = "poetry"  # any entry from GOODREADS_GENRES
MSNEWS_VARIANT = "mind_small_train"  # any entry from MSNEWS_VARIANTS

INTERACTION_LIMIT_OVERRIDE = 180_000  # set to None to use the per-source default
cold_item_frac = 0.2
seed = 42
prefer_gpu = True

k_factors = 16
k_eval = [10, 20, 50]

default_limits = {
    "goodreads": 180_000,
    "amazon": 600_000,
    "movielens": 1_200_000,
    "msnews": 800_000,
}
interaction_limit = (
    INTERACTION_LIMIT_OVERRIDE
    if INTERACTION_LIMIT_OVERRIDE is not None
    else default_limits.get(DATA_SOURCE)
)
# Let MSNews use the full split by default; override above if you truly need a cap.
if DATA_SOURCE == "msnews" and INTERACTION_LIMIT_OVERRIDE is not None:
    interaction_limit = None

DATA_DIR = PROJECT_ROOT / "coldstart" / "data"
if DATA_SOURCE == "amazon":
    DATA_PATH = DATA_DIR / f"amazon_{AMAZON_VARIANT}.csv"
elif DATA_SOURCE == "goodreads":
    DATA_PATH = DATA_DIR / f"goodreads_{GOODREADS_VARIANT}.csv"
elif DATA_SOURCE == "msnews":
    DATA_PATH = DATA_DIR / f"msnews_{MSNEWS_VARIANT}.csv"
else:
    DATA_PATH = DATA_DIR / f"movielens_latest_{DATASET_VARIANT}.csv"

CONFIG_PATH = PROJECT_ROOT / "coldstart" / "configs" / "base.yaml"
config = _read_simple_yaml(CONFIG_PATH)

OUTPUT_ROOT = PROJECT_ROOT / "coldstart" / "output"
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = OUTPUT_ROOT / f"notebook_run_{timestamp}"
RUN_DIR.mkdir(parents=True, exist_ok=True)

print(f"Data path: {DATA_PATH}")
print(f"Run directory: {RUN_DIR}")
if interaction_limit:
    print(f"Capping prepare_dataset to {interaction_limit:,} interactions to conserve RAM.")
else:
    print("Using full dataset (interaction_limit=None). Override above if RAM becomes an issue.")

In [ ]:
if not DATA_PATH.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    if DATA_SOURCE == "amazon":
        df = build_amazon_interaction_frame(dataset=AMAZON_VARIANT, cache_dir=DATA_DIR)
        source_name = f"Amazon {AMAZON_VARIANT}"
    elif DATA_SOURCE == "goodreads":
        df = build_goodreads_interaction_frame(genre=GOODREADS_VARIANT, cache_dir=DATA_DIR)
        source_name = f"Goodreads {GOODREADS_VARIANT}"
    elif DATA_SOURCE == "msnews":
        df = build_msnews_interaction_frame(variant=MSNEWS_VARIANT, cache_dir=DATA_DIR)
        source_name = f"MSNews {MSNEWS_VARIANT}"
    else:
        df = build_interaction_frame(dataset=DATASET_VARIANT)
        source_name = f"MovieLens {DATASET_VARIANT}"
    df.to_csv(DATA_PATH, index=False)
    print(f"Downloaded {source_name} interactions to {DATA_PATH}")
else:
    size_mb = DATA_PATH.stat().st_size / 1e6
    print(f"Dataset already present at {DATA_PATH} (~{size_mb:.1f} MB)")

In [ ]:
tfidf_params = config.get('tfidf', {})
text_encoder_cfg = config.get('text_encoder', {})
encoder_type = text_encoder_cfg.get('type', 'tfidf')
encoder_params = text_encoder_cfg.get('params')
if encoder_params is None and encoder_type == 'tfidf':
    encoder_params = tfidf_params

# MSNews benefits from more text capacity; bump tfidf dims and disable interaction cap.
if DATA_SOURCE == "msnews":
    tfidf_params_local = dict(tfidf_params)
    tfidf_params_local['max_features'] = max(tfidf_params_local.get('max_features', 0) or 0, 5000)
else:
    tfidf_params_local = tfidf_params

prepare_limit = interaction_limit if interaction_limit else None
val_item_frac = float(config.get('prepare', {}).get('val_item_frac', 0) or 0)
val_item_frac = val_item_frac if val_item_frac > 0 else None

pipeline.prepare_dataset(
    DATA_PATH,
    RUN_DIR,
    tfidf_params=tfidf_params_local,
    encoder_type=encoder_type,
    encoder_params=encoder_params if DATA_SOURCE != "msnews" else tfidf_params_local,
    cold_item_frac=cold_item_frac,
    val_item_frac=val_item_frac,
    seed=seed,
    interaction_limit=prepare_limit,
)

sorted_outputs = sorted(path.name for path in RUN_DIR.iterdir())
sorted_outputs

In [ ]:
import copy

cmcl_cfg = copy.deepcopy(dict(config.get('cmcl', {})))
exposure_cfg = dict(cmcl_cfg.get('exposure', {}) or {})
exposure_limit = 350_000
if exposure_cfg.get('max_training_samples') in (None, '', 0):
    exposure_cfg['max_training_samples'] = exposure_limit
cmcl_cfg['exposure'] = exposure_cfg

hard_neg_cfg = dict(cmcl_cfg.get('hard_negatives', {}) or {})
cmcl_cfg['hard_negatives'] = hard_neg_cfg

# For MSNews, give CMCL more capacity/steps.
if DATA_SOURCE == "msnews":
    cmcl_cfg['iters'] = max(int(cmcl_cfg.get('iters', 60)), 120)
    cmcl_cfg['batch_size'] = max(int(cmcl_cfg.get('batch_size', 128)), 256)

print('[cmcl] core settings:')
for key in (
    'lr',
    'reg',
    'iters',
    'batch_size',
    'temperature',
    'self_normalize',
    'max_positives',
    'pi_floor',
    'max_weight',
):
    value = cmcl_cfg.get(key)
    if value is not None:
        print(f"  {key}: {value}")
print('[cmcl] exposure estimator:')
for key, value in cmcl_cfg['exposure'].items():
    print(f"  {key}: {value}")
if hard_neg_cfg.get('k', 0):
    print('[cmcl] hard negatives:')
    for key, value in hard_neg_cfg.items():
        print(f"  {key}: {value}")
else:
    print('[cmcl] hard negatives: disabled (k=0)')

## Train classical baselines
This block fits the matrix-factorization family (Ctrlite, A2F, CTPF, CDL, and HFT) with the prepared splits. CMCL is run separately so we can highlight its contribution.

In [ ]:
mf_cfg = config.get('mf', {})
ctrlite_cfg = config.get('ctrlite', {})
a2f_cfg = config.get('a2f', {})
ctpf_cfg = config.get('ctpf', {})
cdl_cfg = config.get('cdl', {})
hft_cfg = config.get('hft', {})

mf_runtime_cfg = {
    'batch_size': 8192,
    'score_batch_size': 8192,
    'infer_batch_size': 8192,
    'ctrlite_batch_size': 4096,
}

results = pipeline.train_and_evaluate_content_model(
    RUN_DIR,
    k_factors=k_factors,
    k_eval=k_eval,
    mf_reg=float(mf_cfg.get('reg', 0.02)),
    mf_iters=int(mf_cfg.get('iters', 30)),
    mf_lr=float(mf_cfg.get('lr', 0.02)),
    seed=seed,
    ctrlite_reg=float(ctrlite_cfg.get('reg', 0.01)),
    ctrlite_lr=float(ctrlite_cfg.get('lr', 0.1)),
    ctrlite_iters=int(ctrlite_cfg.get('iters', 80)),
    model='ctrlite,a2f,ctpf,cdl,hft',
    a2f_cfg=a2f_cfg,
    ctpf_cfg=ctpf_cfg,
    cdl_cfg=cdl_cfg,
    hft_cfg=hft_cfg,
    cmcl_cfg=cmcl_cfg,
    backend='torch',
    prefer_gpu=prefer_gpu,
    mf_cfg=mf_runtime_cfg,
)
results

## CMCL defaults and ablations
Define a helper to run CMCL with different overrides so we can log the headline model (default) plus the `no_ips_weights` and `random_negatives` ablations mentioned in the paper.

In [ ]:
def _deep_update(base: dict, patch: dict) -> dict:
    for key, value in (patch or {}).items():
        if isinstance(value, dict):
            existing = base.get(key, {})
            if not isinstance(existing, dict):
                existing = {}
            base[key] = _deep_update(existing, value)
        else:
            base[key] = value
    return base


def run_cmcl_variant(label: str, overrides: dict):
    variant_cfg = _deep_update(copy.deepcopy(cmcl_cfg), overrides or {})
    print(f"[cmcl] running {label} with overrides: {overrides}")
    variant_results = pipeline.train_and_evaluate_content_model(
        RUN_DIR,
        k_factors=k_factors,
        k_eval=k_eval,
        mf_reg=float(mf_cfg.get('reg', 0.02)),
        mf_iters=int(mf_cfg.get('iters', 30)),
        mf_lr=float(mf_cfg.get('lr', 0.02)),
        seed=seed,
        ctrlite_reg=float(ctrlite_cfg.get('reg', 0.01)),
        ctrlite_lr=float(ctrlite_cfg.get('lr', 0.1)),
        ctrlite_iters=int(ctrlite_cfg.get('iters', 80)),
        model='cmcl',
        a2f_cfg=a2f_cfg,
        ctpf_cfg=ctpf_cfg,
        cdl_cfg=cdl_cfg,
        hft_cfg=hft_cfg,
        cmcl_cfg=variant_cfg,
        backend='torch',
        prefer_gpu=prefer_gpu,
        mf_cfg=mf_runtime_cfg,
    )
    metrics = variant_results.get('cmcl') if isinstance(variant_results, dict) else variant_results
    if isinstance(metrics, dict):
        hit = metrics.get('hit@10')
        ndcg = metrics.get('ndcg@10')
        if hit is not None and ndcg is not None:
            print(f"[cmcl] {label} hit@10={hit:.4f} ndcg@10={ndcg:.4f}")
    results[f"cmcl_{label}"] = metrics
    return variant_results

In [ ]:
cmcl_default_results = run_cmcl_variant('default', {})

In [ ]:
cmcl_no_ips_results = run_cmcl_variant(
    'no_ips_weights',
    {
        'self_normalize': False,
        'max_weight': 1.0,
        'pi_floor': 1.0,
    },
)

In [ ]:
cmcl_random_neg_results = run_cmcl_variant(
    'random_negatives',
    {
        'hard_negatives': {
            'k': 0,
        },
    },
)

In [ ]:
for path in sorted(RUN_DIR.rglob("*")):
    if path.is_file():
        rel = path.relative_to(PROJECT_ROOT)
        size_kb = path.stat().st_size / 1024
        print(f"{rel} ({size_kb:.1f} KB)")

## Inspect metrics and buckets
Print the nested evaluation dictionary (with confidence intervals, per-bucket metrics, and CMCL variants) using the next cell. Bucket dumps are truncated to keep the Colab log readable.

In [ ]:
import json
from pprint import pprint

pprint(results)
for key, metrics in results.items():
    if key == 'cmcl' or key.startswith('cmcl_'):
        if isinstance(metrics, dict):
            print(f"
{key} bucket snapshot:")
            print(json.dumps(metrics.get('buckets', {}), indent=2)[:2000])